In [1]:
## Import Libraries

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

# File management
import os
import zipfile

# Import relevant libraries
import PIL
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Rescaling
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

%matplotlib inline


2023-01-15 18:30:02.395570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Obtain the dataset 
Considering the size of the dataset, we will remotely obtain the dataset, from kaggle repository.

[Intel Image Classification](https://www.kaggle.com/datasets/puneet6060/intel-image-classification)

## Install Kaggle and set environment

In [2]:
# https://www.kaggle.com/general/74235
# Install Kaggle
!pip install -q kaggle
# create folder ~/.kaggle, which is where kaggle will search for the API Token
!rm -r ~/.kaggle
!mkdir ~/.kaggle
# move kaggle.json to ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Change the permissions of the file. 
!chmod 600 ~/.kaggle/kaggle.json
# Check if it is working
!kaggle datasets list

repository_dir = os.getcwd()

ref                                                             title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ahsan81/hotel-reservations-classification-dataset               Hotel Reservations Dataset                   480KB  2023-01-04 12:50:31           2112         86  1.0              
thedevastator/global-fossil-co2-emissions-by-country-2002-2022  Emissions by Country, 2002-2022              621KB  2023-01-02 20:10:36           1147         42  1.0              
die9origephit/fifa-world-cup-2022-complete-dataset              Fifa World Cup 2022: Complete Dataset          7KB  2022-12-18 22:51:11           5501        177  1.0              
rakkesharv/spotify-top-10000-streamed-songs                     Spotify Top 10000 Streamed Song

## Get dataset

In [3]:
# https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/
zip_file = 'intel-image-classification.zip'
tmp_dir = "dataset"

data_dir = os.path.join(repository_dir, tmp_dir)
if not os.path.exists(data_dir):
    print("create dataset directory")
    os.mkdir(data_dir)
    os.chdir(data_dir)
    print(os.getcwd())
    os.system("kaggle datasets download -d puneet6060/intel-image-classification")
    zip_ref = zipfile.ZipFile(zip_file, 'r')
    zip_ref.extractall(data_dir)
    zip_ref.close()
    os.remove(os.path.join(data_dir, zip_file))
    
train_dir = os.path.join(data_dir, "seg_train", "seg_train")
test_dir = os.path.join(data_dir, "seg_test", "seg_test")

os.chdir(repository_dir)
print(os.getcwd())

/home/renan/Documents/GitHub/FAA_P02


## Loading the images
**Note**: Only load this example for visualization purposes

In [ ]:
validation_percentage = 0.2

data_train, data_val = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=None,
    label_mode="categorical",
    validation_split=validation_percentage,
    subset="both",
    seed=123,
)

data_test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=None,
    label_mode="categorical",
)
class_names = data_train.class_names
print(class_names)

for image, label in data_train.take(1).as_numpy_iterator():
    print(image.shape)
    print(label)
    

## Visualize data

In [ ]:
# from https://www.tensorflow.org/tutorials/load_data/images
import matplotlib.pyplot as plt
subset = list(data_train.take(9).as_numpy_iterator())

fig, axes = plt.subplots(3, 3, figsize=(10,10))
axes = axes.flatten()
for i in range(9):
    ax = axes[i]
    img, label = subset[i]
    label = label.astype("uint8")
    label = np.where(label == 1)[0][0]
    ax.imshow(img.astype("uint8"))
    ax.set_title(class_names[label])
    ax.axis('off')
plt.tight_layout()
plt.show()

## Model number 01 - VGG-16
The VGG-16 is one of the most popular pre-trained models for image classification. Introduced in the famous 
ILSVRC 2014 Conference, it was and remains THE model to beat even today. Developed at the Visual Graphics 
Group at the University of Oxford, VGG-16 beat the then standard of AlexNet and was quickly adopted by 
researchers and the industry for their image Classification Tasks.
https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/


trainable = False

## Reload Image
img_size = (224, 224)
No necessary augmenting data besides image resizing and rescale normalization

In [4]:
img_width = 224
img_height = 224
validation_percentage = 0.2
# Categorical to be able to use category_cross_entropy loss
label_mode = "int"

data_train_vgg16, data_val_vgg16 = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=None,
    label_mode=label_mode,
    color_mode='rgb',
    image_size=(img_width, img_height),
    validation_split=validation_percentage,
    subset="both",
    seed=123,
)

data_test_vgg16 = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=None,
    label_mode=label_mode,
    color_mode='rgb',
    image_size=(img_width, img_height),
)

for image, label in data_train_vgg16.take(1).as_numpy_iterator():
    print(f"image shape: {image.shape}")
    print(label)
    print(f"min value: {np.min(image)}")
    print(f"max value: {np.max(image)}")
    
class_names = data_train_vgg16.class_names
print(class_names)

Found 14034 files belonging to 6 classes.
Using 11228 files for training.
Using 2806 files for validation.


2023-01-15 18:30:19.100544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-15 18:30:19.101688: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 3000 files belonging to 6 classes.
image shape: (224, 224, 3)
2
min value: 14.34375
max value: 255.0
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


## Preprocessing
rescaling = 1.0/255

In [5]:
normalization_layer = layers.Rescaling(1./255)
data_train_vgg16 = data_train_vgg16.map(lambda x,y: (normalization_layer(x), y))

for image, label in data_train_vgg16.take(1).as_numpy_iterator():
    print(f"image shape: {image.shape}")
    print(label)
    print(f"min value: {np.min(image)}")
    print(f"max value: {np.max(image)}")

image shape: (224, 224, 3)
3
min value: 0.0
max value: 1.0


## Creating the model

In [15]:
from tensorflow.keras.applications.vgg16 import VGG16

input_size = (224, 224, 3)

base_model = VGG16(
    input_shape=(224, 224, 3), # Shape of our images
    include_top = False, # Leave out the last fully connected layer
    weights = 'imagenet'
)

# we do not train the parameters
for layer in base_model.layers:
    layer.trainable = False

In [16]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# https://medium.com/analytics-vidhya/car-brand-classification-using-vgg16-transfer-learning-f219a0f09765
# FC layer very simple and with a softmax activation unit
x = layers.Dense(len(class_names), activation="softmax")(x)

landscapeModel01 = Model(inputs=base_model.input, outputs=x, name="landscapeModel01")


In [17]:
loss = "sparse_categorical_crossentropy" #Unvariable, considering the data phenomenum
optimizer = "adam" # Variable

landscapeModel01.compile(
    optimizer=optimizer, 
    loss=loss,
    metrics=["accuracy"]
)


landscapeModel01.summary()


Model: "landscapeModel01"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)  

In [18]:
#fit data
shuffle=True # variable
epochs=50 # variable, according if it is able to converge
batch_size = 200

print(landscapeModel01.input)

landscapeModel01.fit(
    data_train_vgg16,
    validation_data=data_val_vgg16,
    epochs=epochs,
    shuffle=shuffle,
    batch_size=batch_size
)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_5'), name='input_5', description="created by layer 'input_5'")
Epoch 1/50


ValueError: in user code:

    File "/home/renan/anaconda3/envs/faa/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/renan/anaconda3/envs/faa/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/renan/anaconda3/envs/faa/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/renan/anaconda3/envs/faa/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/renan/anaconda3/envs/faa/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/renan/anaconda3/envs/faa/lib/python3.10/site-packages/keras/engine/input_